In [34]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(parallel)
library(ComplexHeatmap)
library(ggsci)
library(tidyverse)

In [35]:
# genes_of_interest <- c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
#                                       'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
#                                       'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND')

In [48]:
genes_of_interest <- c("PTGS2", "BCL2A1", "NLRP3", "OAZ1", "G0S2", "CMTM6", "RASGEF1B", "RIPK2", "RABL6",
                       "SGK1", "SELENOK", "METRNL", "MIDN", "DNAJC10", "BCL6", "PCBP2", "IL1B", "NR4A1", "FKBP5",
                       "SOCS3", "EREG", "TNF", "SHTN1", "WARS", "BAG1", "CDKN2D", "ATP1B3", "RPLP2", "ATP5PD",
                       "CCL3", "CCL4", "CXCL8")

In [49]:
meta_data=read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/scRNA_meta_data-2024-05-09.csv")

In [50]:
colnames(meta_data)

[1] "X"                                        
 [2] "Unnamed..0"                               
 [3] "lastUpdated"                              
 [4] "sample.id"                                
 [5] "sample.bridgingControl"                   
 [6] "sample.sampleKitGuid"                     
 [7] "sample.visitName"                         
 [8] "sample.visitDetails"                      
 [9] "sample.drawDate"                          
[10] "sample.daysSinceFirstVisit"               
[11] "file.id"                                  
[12] "file.name"                                
[13] "file.batchID"                             
[14] "file.panel"                               
[15] "file.pool"                                
[16] "file.fileType"                            
[17] "file.majorVersion"                        
[18] "subject.id"                               
[19] "subject.biologicalSex"                    
[20] "subject.birthYear"                        
[21] "subject.ethnicity"                        
[22] "subject.partnerCode"                      
[23] "subject.race"                             
[24] "subject.subjectGuid"                      
[25] "cohort.cohortGuid"                        
[26] "sample.diseaseStatesRecordedAtVisit"      
[27] "pbmc_sample_id"                           
[28] "subject.covidVaxDose1.daysSinceFirstVisit"
[29] "subject.covidVaxDose2.daysSinceFirstVisit"
[30] "Covid_exclusion"                          
[31] "subjectGuid"                              
[32] "CMV"

In [51]:
# Create a psuebulked object

In [52]:
meta_data_subset=meta_data %>% filter(sample.visitName=="Flu Year 1 Day 0")
aggregated_count_file_list<-paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")

df_list<-mclapply(aggregated_count_file_list,function(x){
df<-read.csv(x,check.names = FALSE, row.names = 1)
colnames(df)<-paste0(gsub("^.*/(.*)\\.csv$", "\\1", x),":",colnames(df))
return (df)    
},mc.cores=60)

In [53]:
unique(meta_data_subset$subject.biologicalSex)

[1] "Female" "Male"

In [54]:
celltype_list <- lapply(df_list, function(df) { df <- df[genes_of_interest, grep("IL1B+ CD14 monocyte", names(df), fixed = TRUE),drop=FALSE] })
exp_matrix<-do.call(cbind,celltype_list)

In [55]:
colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data_subset)<-meta_data_subset$pbmc_sample_id

In [56]:
exp_matrix

,PB00001-01,PB00002-01,PB00003-01,PB00004-01,PB00006-01,PB00010-02,PB00012-01,PB00014-01,PB00015-01,PB00016-01,⋯,PB00592-01,PB00599-01,PB00600-01,PB00602-01,PB00621-01,PB00623-01,PB00625-02,PB00645-01,PB01425-01,PB01446-01
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PTGS2,0.4266934,0.22664239,0.06349906,0.7218853,0.33721047,1.12896432,0.4829994,0.2435899,0.7527992,0.5115903,⋯,0.6651478,0.4964574,0.41292359,0.5533753,1.2982311,0.4040472,1.8283399,0.52041831,0.3277027,0.72567282
BCL2A1,0.9481724,0.76844798,0.96322671,0.5610849,1.03921045,1.64931931,1.2241480,0.7758103,1.0028857,0.9682139,⋯,1.2424633,1.3308464,0.91418128,1.1113097,1.1958907,1.2605281,1.4185560,0.37513346,0.8535119,1.10568353
NLRP3,0.1688363,0.34774061,0.25537076,0.6491407,0.43697227,0.40075560,0.7558058,0.3356846,0.5671532,0.4630318,⋯,0.7700720,0.5915625,0.45056400,0.4961413,0.6239812,0.6803066,0.9365256,0.30951411,0.2541834,0.44944075
OAZ1,2.8916994,3.18063900,2.85717001,2.8435000,3.01100732,3.26351974,2.9924156,2.9788101,2.8032810,3.0899907,⋯,3.0008900,3.0556656,2.97240125,3.1511518,3.0965612,3.0706150,3.4200897,3.02548834,3.0145142,3.08523641
G0S2,0.3337376,0.96725026,0.30764092,0.4807413,0.04824071,0.62082292,0.5757327,0.3791453,0.2909091,0.7263199,⋯,0.8077213,0.5785502,0.28141162,0.3638164,0.0000000,0.7560172,1.2431300,0.35148138,0.3746521,0.14839381
CMTM6,0.9134868,1.08311291,1.14493465,1.1378435,0.86803958,1.04154105,0.7038657,0.7762025,0.7786121,0.9884096,⋯,1.0373951,1.0582421,0.86504783,0.9389128,0.0000000,0.8663381,0.7228292,0.69610102,0.9302463,1.03907224
RASGEF1B,0.3769068,0.17257819,0.52562680,0.4599932,0.35590774,0.70930910,0.5199899,0.1405385,0.7133599,0.4839761,⋯,0.4800682,0.4187579,0.24675981,0.3479129,0.0000000,0.1928807,0.0000000,0.25575745,0.1457756,0.48316907
RIPK2,0.3067567,0.08733411,0.38284983,0.1439108,0.32646699,0.13827821,0.3178129,0.1731436,0.2347619,0.3428669,⋯,0.1764513,0.2304681,0.23119981,0.2018252,0.0000000,0.2910551,0.7228292,0.23408718,0.1286428,0.18754400
RABL6,0.1950485,0.34986142,0.07695097,0.2445314,0.21556647,0.00000000,0.2777071,0.2290301,0.2533447,0.2343692,⋯,0.2752703,0.3829455,0.22820302,0.3073210,0.0000000,0.2200738,0.4820303,0.31416719,0.3846305,0.28636333


In [57]:
write.csv(exp_matrix, "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Figure_Notebooks/Intermediate_files/Mean_gene_expression/IL1B_CD14mono_pseudobulk_genes_expMatrix.csv")